# Barcodes 

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
!pip install python-Levenshtein


In [2]:
import Levenshtein
from Bio import SeqIO

barcodes = '/Users/ariane/Documents/code/MinION/tests/data/20240502/rev_barcodes.fasta'

# Read sequences from a FASTA file
sequences = [str(record.seq) for record in SeqIO.parse(barcodes, "fasta")]

# List of sequence IDs (for labeling purposes)
sequence_ids = [record.id for record in SeqIO.parse(barcodes, "fasta")]


In [4]:
import numpy as np
import Levenshtein

# Create a matrix to store the Levenshtein distances
num_sequences = len(sequences)
distance_matrix = np.zeros((num_sequences, num_sequences), dtype=int)

for i in range(num_sequences):
    for j in range(num_sequences):
        dist = Levenshtein.distance(sequences[i], sequences[j])
        distance_matrix[i, j] = dist
        if dist <= 12 and i != j:
            print(sequence_ids[i], sequence_ids[j], dist)

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(distance_matrix, xticklabels=sequence_ids, yticklabels=sequence_ids, annot=True, cmap='coolwarm')
plt.title("Heatmap of Levenshtein Distance Between Sequences")
plt.show()

In [19]:
len(sequences[0])

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
import Levenshtein
from Bio import SeqIO

barcodes = '/Users/ariane/Documents/code/MinION/tests/data/20240502/for_barcodes.fasta'

# Read sequences from a FASTA file
sequences = [str(record.seq) for record in SeqIO.parse(barcodes, "fasta")]

# List of sequence IDs (for labeling purposes)
sequence_ids = [record.id for record in SeqIO.parse(barcodes, "fasta")]

import numpy as np
import Levenshtein

# Create a matrix to store the Levenshtein distances
num_sequences = len(sequences)
distance_matrix = np.zeros((num_sequences, num_sequences), dtype=int)

for i in range(num_sequences):
    for j in range(num_sequences):
        dist = Levenshtein.distance(sequences[i], sequences[j])
        distance_matrix[i, j] = dist
        if i != j:
            if dist <= 8:
                print(sequence_ids[i], sequence_ids[j], dist)
 
# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(distance_matrix, xticklabels=sequence_ids, yticklabels=sequence_ids, cmap='coolwarm')
plt.title("Heatmap of Levenshtein Distance Between forward barcodes")
plt.show()

In [20]:
8/24

In [7]:
# Create a heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(distance_matrix, xticklabels=sequence_ids, yticklabels=sequence_ids, cmap='coolwarm')
plt.title("Heatmap of Levenshtein Distance Between forward barcodes")
plt.show()

## Testing barcoding

Read layout

Found by regular 3’
-a ADAPTER
Found by non-internal 3’
-a ADAPTERX
Found by anchored 3’
-a ADAPTER$
Full adapter sequence anywhere

acgtacgtADAPTERacgt

yes

no

no

Partial adapter sequence at 3’ end

acgtacgtacgtADAP

yes

yes

no

Full adapter sequence at 3’ end

acgtacgtacgtADAPTER

yes

yes

Example command: 

```
cutadapt -e 8 -a file$:for_barcodes.fasta -o forward.fastq.gz test.fastq.gz
(minion) [tmp] cutadapt -e 8 --revcomp -g ^file:rev_barcode_1.fasta --discard-untrimmed -o test.fastq.gz single_file.fastq.gz

```


In [ ]:
# First cat the fastqz files
# https://cutadapt.readthedocs.io/en/stable/
# (protein) [20240502] cat no_sample/20240502_1704_MN41105_flg114_cc49dc42/fastq_pass/flg114_pass_cc49dc42_c01452a3_* > 20240502.fastq.gz

In [3]:
folder = '/Users/ariane/Documents/code/MinION/manucript/notebooks/Ape AGW/Data/raw/20240502/'
# Test parameters
import os
import time

start = time.time()


barcodes = f'{folder}rev_barcodes.fasta'

# Read sequences from a FASTA file
reverse_barcodes = [str(record.seq) for record in SeqIO.parse(barcodes, "fasta")]
# List of sequence IDs (for labeling purposes)
reverse_barcode_ids = [record.id for record in SeqIO.parse(barcodes, "fasta")]

barcodes = f'{folder}for_barcodes.fasta'

# Read sequences from a FASTA file
forward_barcodes = [str(record.seq) for record in SeqIO.parse(barcodes, "fasta")]
# List of sequence IDs (for labeling purposes)
forward_barcode_ids = [record.id for record in SeqIO.parse(barcodes, "fasta")]

for i, rev in enumerate(reverse_barcodes):
# --discard-untrimmed  only keep adapters with the barcode
    label = reverse_barcode_ids[i]
    print(label)
    os.system(f'cutadapt -e 0.2 -u 200 -O 20 --revcomp -a {rev} --json={label}.cutadapt.json --discard-untrimmed --action=none -o "{folder}output/{label}.fastq.gz" "{folder}20240502.fastq.gz" ')
    os.system(f'mkdir "{folder}output/{label}/"')
    for j, forward in enumerate(forward_barcodes):
        forward_label = forward_barcode_ids[j]
        os.system(f'mkdir "{folder}output/{label}/{forward_label}/"')
        os.system(f'cutadapt -e 0.2 -u 200 -O 20 --revcomp -g {forward} --discard-untrimmed --action=none -o "{folder}output/{label}/{forward_label}/demultiplexed_{label}_{forward_label}.fastq.gz" "{folder}output/{label}.fastq.gz"')

end = time.time()


In [4]:
print(end - start)

In [4]:
from minION import *
vc = VariantCaller(Path('/Users/ariane/Documents/code/MinION/manucript/notebooks/Ape AGW/Data/EVSeqL_Output/YL-EvSeqL1/300-1/'),
                           Path('/Users/ariane/Documents/code/MinION/manucript/notebooks/Ape AGW/Data/EVSeqL_Output/YL-EvSeqL1/300-1/temp_300-1.fasta'),
                           demultiplex_folder_name='',
                           padding_start=0,
                           padding_end=0)
variant_df = vc.get_variant_df(threshold=0.5,
                               min_depth=5,
                               output_dir='/Users/ariane/Documents/code/MinION/manucript/notebooks/Ape AGW/Data/EVSeqL_Output/YL-EvSeqL1/300-1/msa_v2/',
                               num_threads=20)
variant_df.to_csv('/Users/ariane/Documents/code/MinION/manucript/notebooks/Ape AGW/Data/EVSeqL_Output/YL-EvSeqL1/variant_new_0.5_v6_VERSION2.csv')
